In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder


# чтение таблицы симптомов и болезней
df = pd.read_excel('Симпт_бол_врач.xlsx', sheet_name='Симпт_бол_врачи', dtype=object)
df.columns = ['symp', 'diag', 'prof']

# исправление кодировки в поле symp
symp_lst = df['symp'].to_list()
symp_lst = [x.split(' ') for x in symp_lst]
symp_lst = [[x.strip() for x in lst] for lst in symp_lst]
df['symp'] = [' '.join(elem) for elem in symp_lst]

print(df.shape)
df.head(2)

### Кодирование числовыми значениями + словари соотношений

In [10]:
labelencoder = LabelEncoder()
df['diag_n'] = labelencoder.fit_transform(df['diag'].values)
df['symp_n'] = df['symp'].map({value: n for value, n in zip(df['symp'].unique(), range(df['symp'].nunique()))})

symp_codes = {k: v for k, v in zip(df['symp'], df['symp_n'])} # симптомы (текст: код)
diag_codes = {k: v for k, v in zip(df['diag_n'], df['diag'])} # диагнозы (код: текст)

### Создание бинарной матрицы для симптомов

In [11]:
symp_df = df.groupby('diag_n')['symp_n'].apply(set).reset_index()
symp_df = symp_df.sort_values('diag_n').reset_index(drop=True)

symp_df[df['symp_n'].unique()] = 0

for index, row in symp_df.iterrows():
    for value in row['symp_n']:
        symp_df.loc[index, value] = 1

### Обучение

In [12]:
X = symp_df[list(range(df['symp'].nunique()))]
y = symp_df['diag_n']

model = RandomForestClassifier()
model.fit(X, y)

RandomForestClassifier()

### Демонстрация

In [13]:
# словари в помощь
# symp_codes
# diag_codes

In [16]:
# РУЧНОЙ ВВОД
# ввести список симптомов из файла Симпт_бол_врач.xlsx
symptoms_str = ['Боль в ягодице', 
                'Озноб', 
                'Боль в яичнике']

In [21]:
# создание бинарного вектора для списка симптомов
symptoms = [symp_codes[s] for s in symptoms_str]
symptoms_array = list(np.zeros(df['symp'].nunique()))

for s in symptoms:
    symptoms_array.insert(s, 1)
    symptoms_array.pop()
    
symptoms_array = np.array(symptoms_array).reshape(1,-1)

In [23]:
# результат предсказания: таблица диагнозов с вероятностями
res = []
for i, prob in enumerate(model.predict_proba(symptoms_array)[0]):
    if prob != 0:
        _ = [diag_codes[i], prob]
        res.append(_)
        
pd.DataFrame(res, columns=['diag', 'prob']).sort_values('prob', ascending=False)

,diag,prob
16,Поликистоз яичников,0.133000
12,Опухоли яичников,0.127000
13,Остеохондроз поясничный,0.111333
0,Абсцесс мягких тканей,0.091583
17,Фурункул (чирей),0.077083
11,Лимфогранулематоз,0.058310
10,Краснуха,0.055937
9,Коклюш,0.050730
2,Аллергия,0.049294
5,Вирусная инфекция,0.048480
